In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder,StandardScaler,OneHotEncoder
from sklearn.metrics.pairwise import linear_kernel


In [2]:
music_df=pd.read_csv('music_genre.csv')
music_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50005 entries, 0 to 50004
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   instance_id       50000 non-null  float64
 1   artist_name       50000 non-null  object 
 2   track_name        50000 non-null  object 
 3   popularity        50000 non-null  float64
 4   acousticness      50000 non-null  float64
 5   danceability      50000 non-null  float64
 6   duration_ms       50000 non-null  float64
 7   energy            50000 non-null  float64
 8   instrumentalness  50000 non-null  float64
 9   key               50000 non-null  object 
 10  liveness          50000 non-null  float64
 11  loudness          50000 non-null  float64
 12  mode              50000 non-null  object 
 13  speechiness       50000 non-null  float64
 14  tempo             50000 non-null  object 
 15  obtained_date     50000 non-null  object 
 16  valence           50000 non-null  float6

In [3]:
music_df=music_df.dropna()

In [4]:
music_df=music_df[music_df['tempo']!='?']

In [5]:
music_df=music_df[music_df['artist_name']!='empty_field']

In [6]:
music_df

,instance_id,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,obtained_date,valence,music_genre
0,32894.0,Röyksopp,Röyksopp's Night Out,27.0,0.00468,0.652,-1.0,0.941,0.79200,A#,0.115,-5.201,Minor,0.0748,100.889,4-Apr,0.759,Electronic
1,46652.0,Thievery Corporation,The Shining Path,31.0,0.01270,0.622,218293.0,0.890,0.95000,D,0.124,-7.043,Minor,0.0300,115.00200000000001,4-Apr,0.531,Electronic
2,30097.0,Dillon Francis,Hurricane,28.0,0.00306,0.620,215613.0,0.755,0.01180,G#,0.534,-4.617,Major,0.0345,127.994,4-Apr,0.333,Electronic
3,62177.0,Dubloadz,Nitro,34.0,0.02540,0.774,166875.0,0.700,0.00253,C#,0.157,-4.498,Major,0.2390,128.014,4-Apr,0.270,Electronic
4,24907.0,What So Not,Divide & Conquer,32.0,0.00465,0.638,222369.0,0.587,0.90900,F#,0.157,-6.266,Major,0.0413,145.036,4-Apr,0.323,Electronic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50000,58878.0,BEXEY,GO GETTA,59.0,0.03340,0.913,-1.0,0.574,0.00000,C#,0.119,-7.022,Major,0.2980,98.02799999999999,4-Apr,0.330,Hip-Hop
50001,43557.0,Roy Woods,Drama (feat. Drake),72.0,0.15700,0.709,251860.0,0.362,0.00000,B,0.109,-9.814,Major,0.0550,122.04299999999999,4-Apr,0.113,Hip-Hop
50002,39767.0,Berner,Lovin' Me (feat. Smiggz),51.0,0.00597,0.693,189483.0,0.763,0.00000,D,0.143,-5.443,Major,0.1460,131.079,4-Apr,0.395,Hip-Hop
50003,57944.0,The-Dream,Shawty Is Da Shit,65.0,0.08310,0.782,262773.0,0.472,0.00000,G,0.106,-5.016,Minor,0.0441,75.88600000000001,4-Apr,0.354,Hip-Hop


In [7]:
data=music_df.copy().drop(['artist_name','track_name','instance_id','obtained_date','popularity','duration_ms'],axis=1).reset_index(drop=True)

In [8]:
songs=music_df.copy()[['instance_id','artist_name','track_name','music_genre']].reset_index(drop=True)

In [9]:
le=LabelEncoder()
data['key']=le.fit_transform(data['key'])

In [10]:
data['mode']=le.fit_transform(data['mode'])


In [11]:
encoder=OneHotEncoder(sparse=False)
music_genre=encoder.fit_transform(data['music_genre'].values.reshape(-1,1))
music_genre_names=encoder.categories_

c:\Users\tristan\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [12]:
music_genre_names

[array(['Alternative', 'Anime', 'Blues', 'Classical', 'Country',
        'Electronic', 'Hip-Hop', 'Jazz', 'Rap', 'Rock'], dtype=object)]

In [13]:
music_genre_df=pd.DataFrame(columns=music_genre_names,data=music_genre)

In [14]:
music_genre_df

,Alternative,Anime,Blues,Classical,Country,Electronic,Hip-Hop,Jazz,Rap,Rock
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
42786,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
42787,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
42788,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
42789,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [15]:
data=data.drop('music_genre',axis=1)

In [16]:
data=pd.concat([data,music_genre_df],axis=1)

In [17]:
data.columns=data.columns.astype(str)

In [18]:
data

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,...,"('Alternative',)","('Anime',)","('Blues',)","('Classical',)","('Country',)","('Electronic',)","('Hip-Hop',)","('Jazz',)","('Rap',)","('Rock',)"
0,0.00468,0.652,0.941,0.79200,1,0.115,-5.201,1,0.0748,100.889,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.01270,0.622,0.890,0.95000,5,0.124,-7.043,1,0.0300,115.00200000000001,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.00306,0.620,0.755,0.01180,11,0.534,-4.617,0,0.0345,127.994,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.02540,0.774,0.700,0.00253,4,0.157,-4.498,0,0.2390,128.014,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.00465,0.638,0.587,0.90900,9,0.157,-6.266,0,0.0413,145.036,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42786,0.03340,0.913,0.574,0.00000,4,0.119,-7.022,0,0.2980,98.02799999999999,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
42787,0.15700,0.709,0.362,0.00000,2,0.109,-9.814,0,0.0550,122.04299999999999,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
42788,0.00597,0.693,0.763,0.00000,5,0.143,-5.443,0,0.1460,131.079,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
42789,0.08310,0.782,0.472,0.00000,10,0.106,-5.016,1,0.0441,75.88600000000001,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [36]:
songs

,instance_id,artist_name,track_name,music_genre
0,32894.0,Röyksopp,Röyksopp's Night Out,Electronic
1,46652.0,Thievery Corporation,The Shining Path,Electronic
2,30097.0,Dillon Francis,Hurricane,Electronic
3,62177.0,Dubloadz,Nitro,Electronic
4,24907.0,What So Not,Divide & Conquer,Electronic
...,...,...,...,...
42786,58878.0,BEXEY,GO GETTA,Hip-Hop
42787,43557.0,Roy Woods,Drama (feat. Drake),Hip-Hop
42788,39767.0,Berner,Lovin' Me (feat. Smiggz),Hip-Hop
42789,57944.0,The-Dream,Shawty Is Da Shit,Hip-Hop


In [37]:
data.corr()

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,...,"(Alternative,)","(Anime,)","(Blues,)","(Classical,)","(Country,)","(Electronic,)","(Hip-Hop,)","(Jazz,)","(Rap,)","(Rock,)"
acousticness,1.000000,-0.348502,-0.790039,0.387884,0.004335,-0.107498,-0.729521,-0.021100,-0.165749,-0.234088,...,-0.137537,-0.020202,0.009616,0.550374,-0.037003,-0.179023,-0.126494,0.185674,-0.132108,-0.112440
danceability,-0.348502,1.000000,0.264110,-0.299774,-0.016348,-0.052040,0.389825,0.049180,0.282181,-0.015108,...,-0.028247,-0.163714,-0.058192,-0.469612,0.035149,0.113502,0.296715,0.051249,0.259600,-0.038688
energy,-0.790039,0.264110,1.000000,-0.389755,-0.001856,0.185945,0.838255,0.015258,0.158797,0.263184,...,0.138678,0.084533,0.014136,-0.533655,0.048537,0.173515,0.056910,-0.158680,0.064319,0.111165
instrumentalness,0.387884,-0.299774,-0.389755,1.000000,0.000141,-0.091901,-0.527985,0.062053,-0.199019,-0.125927,...,-0.123956,0.100152,-0.089305,0.428229,-0.180665,0.172325,-0.175710,0.178023,-0.176170,-0.131615
key,0.004335,-0.016348,-0.001856,0.000141,1.000000,-0.001814,-0.002252,-0.057731,-0.011682,-0.006764,...,0.006119,0.010150,-0.006929,0.005207,0.013597,-0.005653,-0.005693,0.003145,-0.008377,-0.011484
liveness,-0.107498,-0.052040,0.185945,-0.091901,-0.001814,1.000000,0.124320,-0.014432,0.112828,0.040232,...,0.004381,0.002430,0.079195,-0.065231,-0.015102,0.032979,0.014496,-0.044329,0.005368,-0.013695
loudness,-0.729521,0.389825,0.838255,-0.527985,-0.002252,0.124320,1.000000,0.009731,0.165794,0.239871,...,0.139952,0.065075,0.008123,-0.674432,0.098706,0.111657,0.124238,-0.111479,0.132472,0.104371
mode,-0.021100,0.049180,0.015258,0.062053,-0.057731,-0.014432,0.009731,1.000000,0.077498,-0.020253,...,0.005191,0.015910,-0.049075,-0.017315,-0.160066,0.072791,0.058926,0.080189,0.062841,-0.069303
speechiness,-0.165749,0.282181,0.158797,-0.199019,-0.011682,0.112828,0.165794,0.077498,1.000000,0.068261,...,-0.014433,-0.093513,-0.104106,-0.138987,-0.147268,0.018051,0.373541,-0.063828,0.304043,-0.133941
tempo,-0.234088,-0.015108,0.263184,-0.125927,-0.006764,0.040232,0.239871,-0.020253,0.068261,1.000000,...,0.029271,0.073831,0.015895,-0.172928,0.041658,0.065821,0.000134,-0.090245,0.008000,0.028734


In [19]:
scaler=StandardScaler()
data=scaler.fit_transform(data)

In [20]:
#compute similarities between songs
cosine_sim =linear_kernel(data,data)

In [21]:
cosine_sim.shape

(42791, 42791)

In [22]:
songs

,instance_id,artist_name,track_name,music_genre
0,32894.0,Röyksopp,Röyksopp's Night Out,Electronic
1,46652.0,Thievery Corporation,The Shining Path,Electronic
2,30097.0,Dillon Francis,Hurricane,Electronic
3,62177.0,Dubloadz,Nitro,Electronic
4,24907.0,What So Not,Divide & Conquer,Electronic
...,...,...,...,...
42786,58878.0,BEXEY,GO GETTA,Hip-Hop
42787,43557.0,Roy Woods,Drama (feat. Drake),Hip-Hop
42788,39767.0,Berner,Lovin' Me (feat. Smiggz),Hip-Hop
42789,57944.0,The-Dream,Shawty Is Da Shit,Hip-Hop


In [47]:
song_idx=songs.loc[songs['instance_id']==30097.0].index

In [48]:
rec=pd.DataFrame(cosine_sim[song_idx])
rec=rec.transpose()
score=rec.sort_values(by=0,ascending=False)
similar=songs.iloc[score.index]

In [35]:
similar=songs.iloc[score.index]
similar

,instance_id,artist_name,track_name,music_genre
1897,79863.0,ATLiens,Interstellar - TYNAN Remix,Electronic
3587,25868.0,Leftfield,Afrika Shox,Electronic
4190,89242.0,SLANDER,Superhuman (feat. Eric Leva),Electronic
2594,87417.0,TNGHT,Bugg'n,Electronic
3078,69754.0,Bassnectar,Blast Off,Electronic
...,...,...,...,...
34576,26728.0,Frédéric Chopin,"Waltz in A Minor, B. 150, Op. posth.",Classical
36954,43574.0,Morton Feldman,Rothko Chapel 5,Classical
37272,21527.0,Pascal Rogé,Gymnopedie No.3,Classical
34487,37494.0,Wojciech Kilar,"Moving to the Ghetto Oct. 31, 1940",Classical


In [40]:
best_10=similar.loc[similar['music_genre']=='Electronic'][0:10]

In [45]:
music_df.loc[music_df['instance_id']==best_10.iloc[0]['instance_id']]

,instance_id,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,obtained_date,valence,music_genre
2225,79863.0,ATLiens,Interstellar - TYNAN Remix,26.0,0.0666,0.553,233200.0,0.703,0.00369,G#,0.955,-4.102,Major,0.0702,150.056,4-Apr,0.164,Electronic


In [50]:
music_df.loc[song_idx]

,instance_id,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,obtained_date,valence,music_genre
2,30097.0,Dillon Francis,Hurricane,28.0,0.00306,0.62,215613.0,0.755,0.0118,G#,0.534,-4.617,Major,0.0345,127.994,4-Apr,0.333,Electronic
